# PLR初期設定

PLRのアイテムをJupyter Notebookで処理するためには、本NotebookにしたがってPLR初期設定を実施する必要があります。

# PLRバイナリのインストール

本環境では、PLR CUIバイナリをPLRへのアクセスに使用します。これは公開されていないので、別に共有されたものを **このNotebookと同じディレクトリ** に配置してください。

In [ ]:
import os

assert os.path.exists('plr.zip')

上記のセルがエラーを出力しなければ正常にファイルが設定されている。以下のセルを順番に実行する。

In [ ]:
!unzip -o -d ~/ ./plr.zip

In [ ]:
!mkdir -p ~/.groovy
!cp -fr ~/plr/lib ~/.groovy/

# PLRアカウントの初期設定

以下のCellを1つ1つ実行する。

In [ ]:
import pexpect

plrcommand = os.path.expanduser('~/plr/bin/plr')
plrcui = pexpect.spawn(plrcommand + " storage new googleDrive")

以下のセルを実行すると、URLが表示されるので、ブラウザで開く。ブラウザでGoogle認証を行うと、http://localhost:40385/... というURLが表示されるので、これを入力欄に貼り付ける。(ブラウザ画面にはエラーが表示されるが問題ない)

In [ ]:
plrcui.expect(r'\s+https://accounts.google.com/.*')
print(plrcui.after.decode('utf8').strip())

callback_url = input('このセルの出力に表示されるURLをクリックし、Googleアカウントで認証を行い、ブラウザに表示されたURLを貼り付けてください。')
!curl -v -X GET "{callback_url}"

PLRのパスフレーズを入力する。Personary設定時に指定したパスワードを以下に指定する。

In [ ]:
import re
from getpass import getpass

passphrase = getpass()
plrcui.expect('Enter passphrase:')
storage_id = [re.match(r'.*Storage created:\s*([0-9]+):.*', line).group(1) for line in plrcui.before.decode('utf8').splitlines() if re.match(r'.*Storage created:\s*([0-9]+):.*', line)][0]
plrcui.sendline(passphrase)
plrcui.expect('# ')

storage_id

エラーとならなければ、これで設定は終了。

In [ ]:
plrcui.sendline('exit')
plrcui.expect('Exiting... done.')

得られた設定を設定ファイルに書き出す。

In [ ]:
import os

with open(os.path.expanduser('~/.plrprofile'), 'w') as f:
    f.write('''export STORAGE_ID={storage_id}
export PASSPHRASE="{passphrase}"'''.format(**locals()))

PLRデータ取得サービスを起動する。

In [ ]:
!supervisorctl -s http://localhost:9001 status

In [ ]:
import time

!supervisorctl -s http://localhost:9001 start plrfs
# 起動完了するまで1分待つ
time.sleep(60)

In [ ]:
# 問題が発生した場合は以下のコマンドをコメントアウトして調査する。
# !cat /tmp/supervisor-plrfs.log

# 動作確認

In [ ]:
import asyncio
from plrfs.rpc_client import PLRFSClient

loop = asyncio.get_event_loop()

チャンネル一覧を取得する。

In [ ]:
client = await PLRFSClient(loop).connect()

channels = await client.get_files([])
channels

In [ ]:
# Garmin Connect Channelがなければエラーとなります。無視してください
garmin_channel = [c for c in channels if c['name'] == 'Garmin Connect活動記録'][0]
garmin_channel

In [ ]:
import json

timeline_item_data = []
timeline_items = await client.get_files([garmin_channel['id']])
for item in timeline_items[-20:]:
    timeline_properties = await client.get_files([garmin_channel['id'], item['id']])
    assert 'cnt' in [p['name'] for p in timeline_properties]
    assert 'begin' in [p['name'] for p in timeline_properties]
    assert 'garminJSON' in [p['name'] for p in timeline_properties]
    summary = await client.get_file([garmin_channel['id'], item['id'], [p['id'] for p in timeline_properties if p['name'] == 'cnt'][0]])
    begin = await client.get_file([garmin_channel['id'], item['id'], [p['id'] for p in timeline_properties if p['name'] == 'begin'][0]])
    garminJSON = await client.get_file([garmin_channel['id'], item['id'], [p['id'] for p in timeline_properties if p['name'] == 'garminJSON'][0]])
    timeline_item_data.append((summary['content'].decode('utf8'), begin['content'].decode('utf8'), json.loads(garminJSON['content'])))
#    timeline_item_data
timeline_item_data[-5:]